# Analysing Trail Run Results from cycosports

The data is publicly available from https://cycosports.com/2021-jungle-cross-trail-run-april-3rd-4th/ in PDF format. The table data needs to be extracted from the PDF. We will be analysing the results from 4th April.

https://pdftables.com/ was able to reasonably generate Excel/CSV versions of the datset. The CSV version is used. However, this website only offers 25 pages of free conversions. 

An alternative free site which gives a reasonable output is https://www.pdftoexcelconverter.net/.

# 1. Preparing the data

We will aim to transform this data into a tidy data format as defined by Hadley Wickham (https://en.wikipedia.org/wiki/Tidy_data).

First, we import the necessary libraries and load the DataFrame.

In [1]:
import pandas as pd 
import numpy as np

df = pd.read_csv('results.csv', skiprows=1)

Then, we standardise the format of the column names.

In [2]:
df = df.rename(columns = {"Pl":"category_rank", "overall":"event_rank", "1stLap":"lap_1", "2ndLap":"lap_2"})
df = df.rename(str.lower, axis="columns")

In [3]:
df.head()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time
0,10km - OPEN 13+ YRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Open - Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,Malachy Kirwan (316),NaN,7:16:17.70,21:09.20,21:55.00,43:04.20
4,2.0,2.0,William Petty (267),Coached,7:16:18.70,21:43.80,22:00.40,43:44.20


In [4]:
df.shape

(220, 8)

## Removing the rows with no usable data

* The data contains some rows with the string "Jungle Cross 2021 Trail Run Series Race 2". 
* There are also repeated header rows (with the values "Pl", "overall", "Name", etc.) within the data. This is due to the PDF repeating them for each page. 
* Also, the rows containing "DNS" and "DNF" (in the "Pl" and "overall" data columns) do not have timing information associated with them. 

In [5]:
df[50:55]

,category_rank,event_rank,name,club,start,lap_1,lap_2,time
50,8.0,38.0,Jason Yai (288),NaN,8:28:49.70,30:36.60,31:39.60,1:02:16.20
51,NaN,NaN,Jungle Cross 2021 Trail Run Series Race 2 & 20...,NaN,NaN,NaN,NaN,1
52,Jungle Cross 2021 Trail Run Series Race 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,Pl,overall,Name,Club,Start,1stLap,2ndLap,Time
54,9.0,44.0,Lee Victor (290),NaN,8:28:49.40,32:04.70,32:50.10,1:04:54.80


In [6]:
df.tail()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time
215,Under 14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,DNS DNS,NaN,Hanna Croissant (426),NaN,NaN,NaN,NaN,NaN
217,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,DNS DNS,NaN,Gabriella Faure (417),NaN,NaN,NaN,NaN,NaN
219,NaN,NaN,Jungle Cross 2021 Trail Run Series Race 2 & 20...,NaN,NaN,NaN,NaN,5


We will remove those rows from the DataFrame.

In [7]:
df = df[~df["category_rank"].str.contains("Jungle Cross 2021", na=False, regex=False)]
df = df[~df["name"].str.contains("Jungle Cross 2021", na=False, regex=False)]
df = df[~df["time"].str.contains("Time", na=False, regex=False)]
df = df[~df["category_rank"].str.contains("DNS|DNF", na=False)]

Notes: 

The inversion operator (`~`) is used to return rows not containing the terms. Alternatively, the following can be used:

`df = df[df["column"].str.contains("substring", na=False)==False]`

`regex=False` should not be used if there are regex expressions such as `"DNS|DNF"`.

`na=False` must be used, as the string methods cannot work where values are not a string (e.g. NaN). 
Running the code without `na=False` will result in (for the given column) dropping rows with 
not only the specifed substring, but also those with NaN values.

If we want to verify which rows are being dropped, the code can be run on the original DataFrame but without the inversion, 
to get back a DataFrame with only those rows.

In [8]:
df[50:55]

,category_rank,event_rank,name,club,start,lap_1,lap_2,time
50,8.0,38.0,Jason Yai (288),NaN,8:28:49.70,30:36.60,31:39.60,1:02:16.20
54,9.0,44.0,Lee Victor (290),NaN,8:28:49.40,32:04.70,32:50.10,1:04:54.80
55,10.0,48.0,Daniel Newton (329),NaN,8:28:47.80,30:55.50,34:49.10,1:05:44.60
56,11.0,49.0,Tete Selado (324),NaN,8:27:09.60,30:12.10,35:36.80,1:05:48.90
57,12.0,62.0,Rajen Prabhu (340),Cos coaching,7:19:37.00,35:57.60,39:56.20,1:15:53.80


In [9]:
df.tail()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time
212,4.0,27.0,Janice Teo (419),NaN,12:10:29.10,17:10.20,16:52.00,34:02.20
213,5.0,29.0,Vivi Martanto (423),NaN,12:10:29.60,17:33.90,18:12.70,35:46.60
214,6.0,31.0,Jessica Timms (394),Dulwich Runners,12:09:19.50,17:36.70,21:30.10,39:06.80
215,Under 14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.shape

(182, 8)

## Dealing with the event and gender data in the "category_rank" column

The DataFrame has additional information in the "category_rank" column. The items are: 
* race event (e.g. "10km - Masters (40+)") 
* gender (Male or Female) 
* age category (e.g. Open, Masters, Under 14) in that order. 

They are in their own header rows with no other information. 

This information has to be split out into their own columns to keep the "category_rank" column clean.

### Creating columns for distance, event and age

We can search for rows containing "km", but with first resetting the index to synchronise it with the actual row numbers:

In [11]:
df = df.reset_index(drop=True)
df[df["category_rank"].str.contains("km", na=False, regex=False, case=False)]

,category_rank,event_rank,name,club,start,lap_1,lap_2,time
0,10km - OPEN 13+ YRS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,10km - Masters (40+),NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,3km Adventure Race (7y +),NaN,NaN,NaN,NaN,NaN,NaN,NaN


By running the above code, we can see that there are only 3 race events. Hence, we can use their locations to manually fill a new column via slicing. 

In [12]:
df["race"] = ""
df.loc[:91, "race"] = "10km - Open - 13+"
df.loc[91:142,"race"] = "10km - Masters - 40+"
df.loc[142:,"race"] = "3km - Adventure Race - 7+"

We then remove the now-redundant header rows. We can also split the information in the new column into more granular columns.

In [13]:
df = df.drop([0, 91, 142])

df[["distance","event", "age"]] = df["race"].str.split("-",expand=True)
df = df.drop(columns=["race"])

In [14]:
df.head()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age
1,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10km,Open,13+
2,Open - Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10km,Open,13+
3,1.0,1.0,Malachy Kirwan (316),NaN,7:16:17.70,21:09.20,21:55.00,43:04.20,10km,Open,13+
4,2.0,2.0,William Petty (267),Coached,7:16:18.70,21:43.80,22:00.40,43:44.20,10km,Open,13+
5,3.0,4.0,Chris Timms (251),Dulwich Runners,7:16:58.30,22:15.30,23:27.00,45:42.30,10km,Open,13+


### Creating a column for gender

We can prepare the "gender" column by first making a copy of the "category_rank" column: 

In [15]:
df["gender"] = df["category_rank"]

However, there are 14 header rows for gender as shown below.

In [16]:
len(df[df["gender"].str.fullmatch("Male|Female", na=False)])

14

Hence, we will not use the manual method as shown above, but we will replace all the non-gender values in the column with NaN. This then allows us to forward-fill the values (down the column) from the remaining headers to complete the column.

In [17]:
df["gender"][~df["gender"].str.fullmatch("Male|Female", na=False)] = np.nan
df["gender"] = df.gender.ffill()

In [18]:
df.head()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender
1,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10km,Open,13+,Male
2,Open - Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10km,Open,13+,Male
3,1.0,1.0,Malachy Kirwan (316),NaN,7:16:17.70,21:09.20,21:55.00,43:04.20,10km,Open,13+,Male
4,2.0,2.0,William Petty (267),Coached,7:16:18.70,21:43.80,22:00.40,43:44.20,10km,Open,13+,Male
5,3.0,4.0,Chris Timms (251),Dulwich Runners,7:16:58.30,22:15.30,23:27.00,45:42.30,10km,Open,13+,Male


### Creating a column for category

Again, we can create the column by copying the current "category_rank" column. 

In [19]:
df["category"] = df["category_rank"]

Replacing the rank numbers in the column with NaN values allows us to forward-fill the category information to complete the column.

In [20]:
df["category"] = df["category"].replace(to_replace='\d+\.\d*', value=np.nan, regex=True)
df["category"] = df["category"].ffill()

In [21]:
df.head()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender,category
1,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10km,Open,13+,Male,Male
2,Open - Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10km,Open,13+,Male,Open - Male
3,1.0,1.0,Malachy Kirwan (316),NaN,7:16:17.70,21:09.20,21:55.00,43:04.20,10km,Open,13+,Male,Open - Male
4,2.0,2.0,William Petty (267),Coached,7:16:18.70,21:43.80,22:00.40,43:44.20,10km,Open,13+,Male,Open - Male
5,3.0,4.0,Chris Timms (251),Dulwich Runners,7:16:58.30,22:15.30,23:27.00,45:42.30,10km,Open,13+,Male,Open - Male


Following that, we can strip excess gender information from this column.

In [22]:
df[["category","sex"]] = df["category"].str.split("-",expand=True)
df = df.drop(columns="sex")

In [23]:
df.head()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender,category
1,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10km,Open,13+,Male,Male
2,Open - Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10km,Open,13+,Male,Open
3,1.0,1.0,Malachy Kirwan (316),NaN,7:16:17.70,21:09.20,21:55.00,43:04.20,10km,Open,13+,Male,Open
4,2.0,2.0,William Petty (267),Coached,7:16:18.70,21:43.80,22:00.40,43:44.20,10km,Open,13+,Male,Open
5,3.0,4.0,Chris Timms (251),Dulwich Runners,7:16:58.30,22:15.30,23:27.00,45:42.30,10km,Open,13+,Male,Open


### Removing rows with no time 

At this point, the DataFrame still has the remaining header rows under "category_rank". These rows have no values under the "time" column. 

To clean the data up, we will simply remove all rows with NaN values under "time".

After this, the "category_rank" column should be clean.

In [24]:
df = df.dropna(subset=["time"])
df = df.reset_index(drop=True)

In [25]:
df.head()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender,category
0,1.0,1.0,Malachy Kirwan (316),NaN,7:16:17.70,21:09.20,21:55.00,43:04.20,10km,Open,13+,Male,Open
1,2.0,2.0,William Petty (267),Coached,7:16:18.70,21:43.80,22:00.40,43:44.20,10km,Open,13+,Male,Open
2,3.0,4.0,Chris Timms (251),Dulwich Runners,7:16:58.30,22:15.30,23:27.00,45:42.30,10km,Open,13+,Male,Open
3,4.0,6.0,Benoit Besnier (320),COS Coaching,7:16:17.80,23:32.70,24:02.80,47:35.50,10km,Open,13+,Male,Open
4,5.0,7.0,Daniel Rose (311),Coached Fitness,7:16:59.20,22:55.10,24:41.30,47:36.40,10km,Open,13+,Male,Open


## Dealing with the bib number

The "name" column has additional information regarding the bib number. It is possible to extract that information into a new column and then remove it from the "name" column. 

In [26]:
df["bib_number"] = df["name"].str.extract("\((\d+)\)",expand=True)
df["name"] = df["name"].str.split("(", expand=True)

There are some entries without a bib number, so we will fill those with the string "None". 

In [27]:
df["bib_number"] = df['bib_number'].fillna("None")

In [28]:
df.head()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender,category,bib_number
0,1.0,1.0,Malachy Kirwan,NaN,7:16:17.70,21:09.20,21:55.00,43:04.20,10km,Open,13+,Male,Open,316
1,2.0,2.0,William Petty,Coached,7:16:18.70,21:43.80,22:00.40,43:44.20,10km,Open,13+,Male,Open,267
2,3.0,4.0,Chris Timms,Dulwich Runners,7:16:58.30,22:15.30,23:27.00,45:42.30,10km,Open,13+,Male,Open,251
3,4.0,6.0,Benoit Besnier,COS Coaching,7:16:17.80,23:32.70,24:02.80,47:35.50,10km,Open,13+,Male,Open,320
4,5.0,7.0,Daniel Rose,Coached Fitness,7:16:59.20,22:55.10,24:41.30,47:36.40,10km,Open,13+,Male,Open,311


## Removing NaN values from the "club" column

For consistency, we will do the same for NaN values in the "club" column as we did for the "bib_number" column.

In [29]:
df['club'] = df['club'].fillna("None")
df.head()

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender,category,bib_number
0,1.0,1.0,Malachy Kirwan,None,7:16:17.70,21:09.20,21:55.00,43:04.20,10km,Open,13+,Male,Open,316
1,2.0,2.0,William Petty,Coached,7:16:18.70,21:43.80,22:00.40,43:44.20,10km,Open,13+,Male,Open,267
2,3.0,4.0,Chris Timms,Dulwich Runners,7:16:58.30,22:15.30,23:27.00,45:42.30,10km,Open,13+,Male,Open,251
3,4.0,6.0,Benoit Besnier,COS Coaching,7:16:17.80,23:32.70,24:02.80,47:35.50,10km,Open,13+,Male,Open,320
4,5.0,7.0,Daniel Rose,Coached Fitness,7:16:59.20,22:55.10,24:41.30,47:36.40,10km,Open,13+,Male,Open,311


Now, we have values in all cells of the DataFrame.

In [30]:
df.isna().sum().sum()

0

## Properly formatting the duration-based columns

The "lap_1", "lap_2" and "time" columns contain duration information. Analysis requires conversion to the Pandas timedelta object. However, using the columns as they are will throw errors with `pd.to_timedelta`. 

Within the same column, some entries are in `%M:%S.%f`(MM:SS:ff) and some are in `%-H:%M:%S.%f` (H:MM:SS:ff) format. 

In [31]:
df[60:65]

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender,category,bib_number
60,1.0,27.0,Luu Phuong Anh Dang,None,8:27:09.30,27:50.50,28:51.00,56:41.50,10km,Open,13+,Female,Open,None
61,2.0,28.0,Emily Astiz,Team Grit OCR,8:26:19.80,27:38.60,29:23.90,57:02.50,10km,Open,13+,Female,Open,298
62,3.0,36.0,Michelle Ferguson,None,8:26:19.60,29:21.70,31:49.20,1:01:10.90,10km,Open,13+,Female,Open,303
63,4.0,45.0,Véronique Gille,None,7:20:35.00,32:43.00,32:26.50,1:05:09.50,10km,Open,13+,Female,Open,281
64,5.0,47.0,Rachel Halliday,Dulwich Runners,8:28:03.90,32:38.70,33:02.00,1:05:40.70,10km,Open,13+,Female,Open,254


As long as we change the `%M:%S.%f` entries to `%-H:%M:%S.%f` so that the whole column is consistent, `pd.to_timedelta` will accept it. 

However, we will change both the `%M:%S.%f` and the `%-H:%M:%S.%f` entries in the column to `%H:%M:%S.%f` (HH:MM:SS:ff) as it is more conventional. 

There are three ways we can write functions to update the formats. The functions will add the respective zero digits and semicolons to the strings where applicable. 

### Method 1: Counting the number of ":" in the strings

In [32]:
def add_hours_zero(column):
    m = column.str.count(':') == 2
    column = column.mask(m, "0" + column, axis=0)
    return column

def add_hours(column):
    m = column.str.count(':') == 1
    column = column.mask(m, "00:" + column, axis=0)
    return column

### Method 2: Measuring the length of the string

In [33]:
def add_hours_zero(column):
    m = column.str.len() == 10
    column = column.mask(m, "0" + column, axis=0)
    return column

def add_hours(column):
    m = column.str.len() == 8
    column = column.mask(m, "00:" + column, axis=0)
    return column  

### Method 3: Using regex

In [34]:
def add_hours_zero(column):
    m = column.str.contains("^\d+:\d+:\d+\.\d+$", na=False)
    column = column.mask(m, "0" + column, axis=0)
    return column

def add_hours(column):
    m = column.str.contains("^\d+:\d+\.\d+$", na=False)
    column = column.mask(m, "00:" + column, axis=0)
    return column

### Applying the functions

Note that `add_hours_zero` must be applied before `add_hours` to work properly on the data.

In [35]:
df[["lap_1", "lap_2", "time"]] = df[["lap_1", "lap_2", "time"]].apply(add_hours_zero)
df[["lap_1", "lap_2", "time"]] = df[["lap_1", "lap_2", "time"]].apply(add_hours)

In [36]:
df[60:65]

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender,category,bib_number
60,1.0,27.0,Luu Phuong Anh Dang,None,8:27:09.30,00:27:50.50,00:28:51.00,00:56:41.50,10km,Open,13+,Female,Open,None
61,2.0,28.0,Emily Astiz,Team Grit OCR,8:26:19.80,00:27:38.60,00:29:23.90,00:57:02.50,10km,Open,13+,Female,Open,298
62,3.0,36.0,Michelle Ferguson,None,8:26:19.60,00:29:21.70,00:31:49.20,01:01:10.90,10km,Open,13+,Female,Open,303
63,4.0,45.0,Véronique Gille,None,7:20:35.00,00:32:43.00,00:32:26.50,01:05:09.50,10km,Open,13+,Female,Open,281
64,5.0,47.0,Rachel Halliday,Dulwich Runners,8:28:03.90,00:32:38.70,00:33:02.00,01:05:40.70,10km,Open,13+,Female,Open,254


## Formatting the "start" column to datetime format 

The "start" column contains the race start time of the runner on that day. After formatting it to datetime format, year-month-day placeholders appear in the entries. We can specify the actual date of the event.

In [37]:
df["start"] = pd.to_datetime(df["start"], format="%H:%M:%S.%f")
df["start"] = df["start"].map(lambda x: x.replace(year=2021, month=4, day=4))

## Casting of types for the rank columns 

The "category_rank" and "event_rank" columns are of dtype: object and appear formatted as a float in the dataframe. We can change them to integers.

In [38]:
df["category_rank"] = pd.to_numeric(df["category_rank"]).astype(int)
df["event_rank"] = pd.to_numeric(df["event_rank"]).astype(int)

In [39]:
df.dtypes

category_rank             int32
event_rank                int32
name                     object
club                     object
start            datetime64[ns]
lap_1                    object
lap_2                    object
time                     object
distance                 object
event                    object
age                      object
gender                   object
category                 object
bib_number               object
dtype: object

## Exporting the cleaned data to .csv 

Now, we can export the data, and re-import it to validate it.

In [40]:
df.to_csv("results_clean.csv", index=False )
df = pd.read_csv('results_clean.csv')

In [41]:
df

,category_rank,event_rank,name,club,start,lap_1,lap_2,time,distance,event,age,gender,category,bib_number
0,1,1,Malachy Kirwan,None,2021-04-04 07:16:17.700,00:21:09.20,00:21:55.00,00:43:04.20,10km,Open,13+,Male,Open,316
1,2,2,William Petty,Coached,2021-04-04 07:16:18.700,00:21:43.80,00:22:00.40,00:43:44.20,10km,Open,13+,Male,Open,267
2,3,4,Chris Timms,Dulwich Runners,2021-04-04 07:16:58.300,00:22:15.30,00:23:27.00,00:45:42.30,10km,Open,13+,Male,Open,251
3,4,6,Benoit Besnier,COS Coaching,2021-04-04 07:16:17.800,00:23:32.70,00:24:02.80,00:47:35.50,10km,Open,13+,Male,Open,320
4,5,7,Daniel Rose,Coached Fitness,2021-04-04 07:16:59.200,00:22:55.10,00:24:41.30,00:47:36.40,10km,Open,13+,Male,Open,311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2,23,Anupama Prakash,None,2021-04-04 12:09:54.900,00:12:50.20,00:13:45.30,00:26:35.50,3km,Adventure Race,7+,Female,Open,407
141,3,25,Vanessa Routley,None,2021-04-04 12:09:50.800,00:13:24.40,00:17:48.20,00:31:12.60,3km,Adventure Race,7+,Female,Open,408
142,4,27,Janice Teo,None,2021-04-04 12:10:29.100,00:17:10.20,00:16:52.00,00:34:02.20,3km,Adventure Race,7+,Female,Open,419
143,5,29,Vivi Martanto,None,2021-04-04 12:10:29.600,00:17:33.90,00:18:12.70,00:35:46.60,3km,Adventure Race,7+,Female,Open,423


In [42]:
df.isna().sum().sum()

0

--End of Part 1--